<a href="https://colab.research.google.com/github/Arseniy-Polyakov/machine_learning_course/blob/main/Task_4_RNN_ipynb.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Данная работа посвящена созданию переводчика с карельского языка на русский. Актуальность выбора работы можно подтвердить развитием ревитализации и сохранения миноритарных языков России в контексте языковой политики, в частности карельского языка, который имеет III тип (языком владеет только старшее поколение, язык не передается молодому поколению) по классификации витальности языков В.М. Алпатова.

Устанавливаем библиотеки для работы с RNN

In [1]:
!pip install tensorflow keras

Импортируем модули для анализа датасетов и обучения модели

In [72]:
import re
import random
import keras
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu, SmoothingFunction

В качестве обучающей выборки будет рассмотрен параллельный подкорпус НКРЯ и самое известное и полномасштабное произведение из карельского эпоса, на основе которого создаются словари и другие лексикографические источники, [Калевала](https://ru.wikipedia.org/wiki/%D0%9A%D0%B0%D0%BB%D0%B5%D0%B2%D0%B0%D0%BB%D0%B0)

Соберем данные из параллельного подкорпуса НКРЯ

In [84]:
file_name = "НКРЯ.xlsx"
df = pd.read_excel(file_name)

df_parallel = df[["Full context", "Para context 1"]]
df_parallel.columns = ["Russian", "Karelian"]

russian_texts_nkrya_raw = list(df_parallel["Russian"])
karelian_texts_nkrya_raw = list(df_parallel["Karelian"])

print(russian_texts_nkrya_raw[:20])
print(karelian_texts_nkrya_raw[:20])

['А что говорят?  Здесь был дом, вот здесь был дом.', 'А что говорят?  Здесь был дом, вот здесь был дом.', 'А что говорят?  Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.', 'Здесь был дом, вот здесь был дом.  Домов было много.  Но, это самое, вот здесь был дом, жили Филипповы.', 'Здесь был дом, вот здесь был дом.  Домов было много.  Но, это самое, вот здесь был дом, жили Филипповы.', 'Здесь был дом, вот здесь был дом.  Домов было много.  Но, это самое, вот здесь был дом, жили Филипповы.', 'Домов было много.  Но, это самое, вот здесь был дом, жили Филипповы.  Потом был еще этот, Зорькины жили, был дом.', 'Домов было много.  Но, это самое, вот здесь был дом, жили Филипповы.  Потом был еще этот, Зорькины жили, был дом.', 'Домов было много.  Но, эт

Функции препроцессинга текста

In [4]:
def preprocessing_russian(texts_russian: str) -> str:
  russian_texts_cleaned = re.sub(r"[^а-яё\s\-]", "", texts_russian.lower())
  return russian_texts_cleaned

def preprocessing_karelian(texts_karelian: str) -> str:
  karelian_texts_cleaned = re.sub(r"[^A-Za-zČčŠšŽžÄäÖöʼ\s]", "", texts_karelian.lower())
  karelian_texts_cleaned = karelian_texts_cleaned.replace("<br/>", "")
  return karelian_texts_cleaned

Обработанные тексты из подкорпуса НКРЯ

In [85]:
russian_texts_nkrya = [preprocessing_russian(text).strip() for text in russian_texts_nkrya_raw]
karelian_texts_nkrya = [preprocessing_karelian(text).strip() for text in karelian_texts_nkrya_raw]

print(russian_texts_nkrya[:20])
print(karelian_texts_nkrya[:20])

['а что говорят  здесь был дом вот здесь был дом', 'а что говорят  здесь был дом вот здесь был дом', 'а что говорят  здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом', 'здесь был дом вот здесь был дом  домов было много  но это самое вот здесь был дом жили филипповы', 'здесь был дом вот здесь был дом  домов было много  но это самое вот здесь был дом жили филипповы', 'здесь был дом вот здесь был дом  домов было много  но это самое вот здесь был дом жили филипповы', 'домов было много  но это самое вот здесь был дом жили филипповы  потом был еще этот зорькины жили был дом', 'домов было много  но это самое вот здесь был дом жили филипповы  потом был еще этот зорькины жили был дом', 'домов было много  но это самое вот здесь был дом жили филипповы  потом был еще этот з

Из готовых параллельных корпусов карельского и русского языков есть только параллельный подкорпус НКРЯ, поэтому был самостоятельно собран и обработан параллельный корпус на основе произведения Калевала, для того чтобы расширить обучающую выборку

In [6]:
with open("Калевала.txt", "rt", encoding="utf-8") as file:
  text = file.read()
text_preprocessed = re.sub(r"[\.\!\?,\d\;\:\*—]", "", text.lower())
text_splitted = re.split(r"\t|\n|/td|>", text_preprocessed)
texts_cleaned = [sentence for sentence in text_splitted if sentence.isspace() == False and len(sentence) > 0]

karelian_texts_kalevala = [preprocessing_karelian(texts_cleaned[i]) for i in range(len(texts_cleaned)) if texts_cleaned[i].isspace() == False and i % 2 == 0]
russian_texts_kalevala = [preprocessing_russian(texts_cleaned[i]) for i in range(len(texts_cleaned)) if texts_cleaned[i].isspace() == False and i % 2 != 0]

# Сохранение обработанного корпуса
with open("параллельный корпус.txt", "wt", encoding="utf-8") as file:
  for i in range(len(karelian_texts_kalevala)):
    file.write(karelian_texts_kalevala[i])
    file.write("\n")
    file.write(russian_texts_kalevala[i])
    file.write("\n")

# Проверка на правильное соотношение пар (возможна благодаря разной письменности языков)
check_karelian = [re.sub(r"[^А-Яа-яёЁ]", "", sentence) for sentence in karelian_texts_kalevala]
check_russian = [re.sub(r"[^A-Za-zČčŠšŽžÄäÖö]", "", sentence) for sentence in russian_texts_kalevala]

print(karelian_texts_kalevala)
print(russian_texts_kalevala)


['mieleni minun tekevi', 'aivoni ajattelevi', 'lähteäni laulamahan', 'saaani sanelemahan', 'sukuvirttä suoltamahan', 'lajivirttä laulamahan', 'sanat suussani sulavat', 'puheet putoelevat', 'kielelleni kerkiävät', 'hampahilleni hajoovat', 'veli kulta veikkoseni', 'kaunis kasvinkumppalini', 'lähe nyt kanssa laulamahan', 'saa kera sanelemahan', 'yhtehen yhyttyämme', 'kahtaalta käytyämme', 'harvoin yhtehen yhymme', 'saamme toinen toisihimme', 'näillä raukoilla rajoilla', 'poloisilla pohjan mailla', 'lyökämme käsi kätehen', 'sormet sormien lomahan', 'lauloaksemme hyviä', 'parahia pannaksemme', 'kuulla noien kultaisien', 'tietä mielitehtoisien', 'nuorisossa nousevassa', 'kansassa kasuavassa', 'noita saamia sanoja', 'virsiä virittämiä', 'vyöltä vanhan väinämöisen', 'alta ahjon ilmarisen', 'päästä kalvan kaukomielen', 'joukahaisen jousen tiestä', 'pohjan peltojen periltä', 'kalevalan kankahilta', 'niit ennen isoni lauloi', 'kirvesvartta vuollessansa', 'niitä äitini opetti', 'väätessänsä värtti

Объединим два корпуса для обучающей выборки

In [7]:
russian_texts_overall = russian_texts_nkrya + russian_texts_kalevala
karelian_texts_overall = karelian_texts_nkrya + karelian_texts_kalevala

russian_texts_input = ["startseq " + text for text in russian_texts_overall]
russian_texts_output = [text + " endseq" for text in russian_texts_overall]

Проводим токенизацию текста и создаем словари

In [8]:
karelian_tokenizer = Tokenizer(filters="")
karelian_tokenizer.fit_on_texts(karelian_texts_overall)
karelian_seqs = karelian_tokenizer.texts_to_sequences(karelian_texts_overall)
max_karelian_len = max(len(seq) for seq in karelian_seqs)
karelian_seqs = pad_sequences(karelian_seqs, maxlen=max_karelian_len, padding="post")

rus_tokenizer = Tokenizer(filters="")
rus_tokenizer.fit_on_texts(russian_texts_input + russian_texts_output)
rus_seqs_in = rus_tokenizer.texts_to_sequences(russian_texts_input)
rus_seqs_out = rus_tokenizer.texts_to_sequences(russian_texts_output)
max_rus_len = max(len(seq) for seq in rus_seqs_in)
rus_seqs_in = pad_sequences(rus_seqs_in, maxlen=max_rus_len, padding="post")
rus_seqs_out = pad_sequences(rus_seqs_out, maxlen=max_rus_len, padding="post")

karelian_vocab_size = len(karelian_tokenizer.word_index) + 1
rus_vocab_size = len(rus_tokenizer.word_index) + 1

rus_seqs_out = np.expand_dims(rus_seqs_out, -1)

Задаем параметры модели

In [9]:
embedding_dim = 64
lstm_units = 128

Создаем энкодер и декодер

In [36]:
encoder_inputs = Input(shape=(max_karelian_len,))
enc_emb = Embedding(karelian_vocab_size, embedding_dim, mask_zero=True)(encoder_inputs)
encoder_lstm = LSTM(lstm_units, return_state=True, use_cudnn=False)
_, state_h, state_c = encoder_lstm(enc_emb)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(max_rus_len,))
dec_emb_layer = Embedding(rus_vocab_size, embedding_dim, mask_zero=True)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(lstm_units, return_sequences=True, return_state=True, use_cudnn=False)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(rus_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Обучение модели

In [37]:
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.fit([karelian_seqs, rus_seqs_in], rus_seqs_out, batch_size=64, epochs=50, verbose=0)

Создадим инференс модели

In [39]:
encoder_model_inference = Model(encoder_inputs, encoder_states)
decoder_state_input_h = Input(shape=(lstm_units,))
decoder_state_input_c = Input(shape=(lstm_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
dec_emb_inference = dec_emb_layer(decoder_inputs)
decoder_outputs_inf, state_h_inf, state_c_inf = decoder_lstm(
  dec_emb_inference, initial_state=decoder_states_inputs)
decoder_outputs_inf = decoder_dense(decoder_outputs_inf)
decoder_states_inf = [state_h_inf, state_c_inf]

decoder_model_inference = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs_inf] + decoder_states_inf
)

Функция для перевода с карельского языка на русский

In [40]:
def karelian_to_russian(input_text: str, max_decoder_seq_length=max_rus_len) -> str:
    input_seq = karelian_tokenizer.texts_to_sequences([input_text])
    input_seq = pad_sequences(input_seq, maxlen=max_karelian_len, padding='post')

    states_value = encoder_model_inference.predict(input_seq)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = rus_tokenizer.word_index['startseq']

    decoded_sentence = []
    for _ in range(max_decoder_seq_length):
        output_tokens, h, c = decoder_model_inference.predict([target_seq] + states_value)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = rus_tokenizer.index_word.get(sampled_token_index, '')

        if sampled_word == 'endseq' or sampled_word == '':
            break
        decoded_sentence.append(sampled_word)

        target_seq[0, 0] = sampled_token_index
        states_value = [h, c]

    return ' '.join(decoded_sentence)

Для оценки качества модели будем использовать sentence bleu score с параметром Smoothing (method4). Определим рандомным (псевдорандомным) методом 20 предложений для перевода, посчитаем sentence bleu score для каждого и найдем среднее арифметическое

In [83]:
random_indexes = [random.randint(i, len(russian_texts_overall)) for i in range(20)]
karelian_russian_texts_random = [(karelian_texts_overall[index], russian_texts_overall[index]) for index in random_indexes]
translations = [(karelian_to_russian(sentence[0]), sentence[1]) for sentence in karelian_russian_texts_random]
translations
bleu_sentence_scores = [sentence_bleu([sentence[0]], sentence[1], smoothing_function=SmoothingFunction().method4) for sentence in translations]
bleu_sentence_scores_average = round(sum(bleu_sentence_scores) / len(bleu_sentence_scores), 2)
bleu_sentence_scores_average

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 39ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 36ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━

0.32

Дальнейшее улучшение модели:
1. Пополнить параллельный корпус другими данными (произведения, учебные тексты на карельском языке)
2. Использовать алгоритм внимания (attention)
3. Расширить пул метрик оценивания модели